<a href="https://colab.research.google.com/github/umabhargavi20/Robustness-Evaluation-of-CNN-ResNet-and-CLIP-Models-Against-Adversarial-Attacks/blob/main/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/TML/ATTACKS_CIFAR_MNIST')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K

In [ ]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()



11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
x_train = x_train.reshape(x_train.shape[0],28,28,1)
x_test = x_test.reshape(x_test.shape[0],28,28,1)

In [ ]:
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [ ]:
y_train =to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(32, kernel_size = (3,3),activation = 'relu', input_shape = (28, 28, 1)  ))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.add(MaxPooling2D(pool_size = (2,2)))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(128,activation = 'relu'))
model.add(Dense(10,activation = 'softmax'))

In [ ]:
from tensorflow.keras.losses import categorical_crossentropy
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 34s 69ms/step - accuracy: 0.8836 - loss: 0.4401 - val_accuracy: 0.9717 - val_loss: 0.0934
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 30s 65ms/step - accuracy: 0.9772 - loss: 0.0755 - val_accuracy: 0.9811 - val_loss: 0.0541
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 65ms/step - accuracy: 0.9864 - loss: 0.0472 - val_accuracy: 0.9832 - val_loss: 0.0487
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 40s 64ms/step - accuracy: 0.9898 - loss: 0.0344 - val_accuracy: 0.9868 - val_loss: 0.0432
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 64ms/step - accuracy: 0.9926 - loss: 0.0259 - val_accuracy: 0.9849 - val_loss: 0.0457
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 64ms/step - accuracy: 0.9949 - loss: 0.0184 - val_accuracy: 0.9869 - val_loss: 0.0443
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 64ms/step - accuracy: 0.9962 - loss: 0.0134 - val_accuracy: 0.9862 - val_loss: 0.0428
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 30s 64ms/step - accuracy: 0.9967 - loss: 0.0110 - 

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', test_accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9808 - loss: 0.0619
Test accuracy: 0.9851999878883362


In [ ]:
model.save('./model/MNIST_model.keras')

In [ ]:
loaded_model = load_model('./model/MNIST_model.keras')

In [ ]:
# Step 1: Randomly select 10% of the test set
num_images = x_test.shape[0]
num_attack_images = int(num_images * 0.1)  # 10% of the test set

# Randomly select indices for 10% of the images
attack_indices = np.random.choice(num_images, num_attack_images, replace=False)

# Step 2: Create a copy of x_test to hold the adversarial images
x_test_adv = np.copy(x_test)



In [ ]:

# Step 5: FGSM Attack Implementation
def fgsm_attack(model, x, y, epsilon=0.1):
    x_var = tf.Variable(x)
    with tf.GradientTape() as tape:
        tape.watch(x_var)
        predictions = model(x_var)
        loss = tf.keras.losses.categorical_crossentropy(y, predictions)
    gradient = tape.gradient(loss, x_var)
    perturbation = epsilon * tf.sign(gradient)
    x_adv = x + perturbation
    x_adv = tf.clip_by_value(x_adv, 0, 1)  # Ensure the values are between 0 and 1
    return x_adv

In [ ]:
# Step 3: Apply the FGSM attack only to the selected 10% of images
x_test_adv[attack_indices] = fgsm_attack(model, x_test[attack_indices], y_test[attack_indices], epsilon=0.5)

In [ ]:
# Step 7: Evaluate on adversarial examples
loss_adv, accuracy_adv = model.evaluate(x_test_adv, y_test)
print(f"Test Loss (Adversarial): {loss_adv}")
print(f"Test Accuracy (Adversarial): {accuracy_adv}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1203 - loss: 20.9548
Test Loss (Adversarial): 20.93642807006836
Test Accuracy (Adversarial): 0.12200000137090683


In [ ]:

# PGD Attack Function
def pgd_attack(model, x, y, epsilon=0.1, alpha=0.01, num_iter=40):
    """
    Perform PGD attack on input data.

    Args:
        model: The target model.
        x: Input data (images).
        y: True labels.
        epsilon: Maximum perturbation.
        alpha: Step size for each iteration.
        num_iter: Number of iterations.

    Returns:
        Adversarial examples.
    """
    # Start with the original image
    x_adv = tf.Variable(x)

    for _ in range(num_iter):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            predictions = model(x_adv)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)

        # Get the gradient of the loss with respect to the input image
        gradient = tape.gradient(loss, x_adv)

        # Apply the perturbation (sign of the gradient * step size)
        perturbation = alpha * tf.sign(gradient)

        # Update the adversarial image by adding the perturbation
        x_adv = x_adv + perturbation

        # Project the adversarial image back into the valid range [0, 1]
        x_adv = tf.clip_by_value(x_adv, 0, 1)

        # Ensure the perturbation is within the epsilon bound
        perturbation = tf.clip_by_value(x_adv - x, -epsilon, epsilon)
        x_adv = x + perturbation

        # Ensure the values are within [0, 1]
        x_adv = tf.clip_by_value(x_adv, 0, 1)

    return x_adv

In [ ]:
# Step 3: Apply the PGD attack only to the selected 10% of images
for i in attack_indices:
    x_test_adv[i] = pgd_attack(model, x_test[i:i+1], y_test[i:i+1], epsilon=0.1, alpha=0.01, num_iter=40)

In [ ]:
# Step 4: Evaluate the model on the entire (mixed) x_test dataset
loss, accuracy = model.evaluate(x_test_adv, y_test)
print(f"Test Loss (with 10% poisoned by PGD): {loss}")
print(f"Test Accuracy (with 10% poisoned by PGD): {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1982 - loss: 6.0058
Test Loss (with 10% poisoned by PGD): 5.818140506744385
Test Accuracy (with 10% poisoned by PGD): 0.2053000032901764


In [ ]:
import random
from scipy.ndimage import shift

# Spatial Attack Function (Random Pixel Shifts)
def spatial_attack(x, shift_range=5):
    """
    Apply a spatial attack by shifting pixels in random directions.

    Args:
        x: Input image (1 sample).
        shift_range: Range for random pixel shifts (how many pixels to shift).

    Returns:
        Adversarial example.
    """
    # Randomly shift the image in both x and y directions
    shift_x = random.randint(-shift_range, shift_range)
    shift_y = random.randint(-shift_range, shift_range)

    # Apply the shift to the image
    x_adv = shift(x, (shift_x, shift_y, 0), mode='nearest')

    # Ensure the image is within [0, 1]
    x_adv = np.clip(x_adv, 0, 1)

    return x_adv

In [ ]:
# Step 3: Apply the Spatial attack only to the selected 10% of images
for i in attack_indices:
    x_test_adv[i] = spatial_attack(x_test[i], shift_range=5)

In [ ]:
# Step 4: Evaluate the model on the entire (mixed) x_test dataset
loss, accuracy = model.evaluate(x_test_adv, y_test)
print(f"Test Loss (with 10% poisoned by Spatial Attack): {loss}")
print(f"Test Accuracy (with 10% poisoned by Spatial Attack): {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9300 - loss: 0.5059
Test Loss (with 10% poisoned by Spatial Attack): 0.502133846282959
Test Accuracy (with 10% poisoned by Spatial Attack): 0.9343000054359436
